In [ ]:
# caregiver class
class Caregiver:
    def __init__(self, name, caregiver_type, availability, phone=None, email=None, pay_rate=20):
        self.name = name
        self.caregiver_type = caregiver_type
        self.availability = availability  # Example: {1: {"7AM-1PM": "preferred", "1PM-7PM": "available"}}
        self.phone = phone
        self.email = email
        self.pay_rate = pay_rate
        self.hours_assigned = 0  # To calculate weekly pay

    def update_availability(self, day, shift, status):
        """
        Updates the availability for a given day and shift.
        """
        if day not in self.availability:
            self.availability[day] = {}
        self.availability[day][shift] = status

    def calculate_weekly_pay(self):
        """
        Calculates the weekly pay based on assigned hours.
        """
        return self.hours_assigned * self.pay_rate


# schedule class
class Schedule:
    def __init__(self, caregivers):
        self.caregivers = caregivers
        self.schedule = {}

    def initialize_schedule(self, days_in_month=31):
        """
        Initializes the schedule for each day of the month with two shifts (AM and PM).
        """
        for day in range(1, days_in_month + 1):
            self.schedule[day] = {"7AM-1PM": None, "1PM-7PM": None}

    def assign_caregivers(self):
        """
        Assigns caregivers to shifts based on their availability.
        """
        for day, shifts in self.schedule.items():
            for shift in shifts:
                # Get caregivers with 'preferred' availability first
                preferred_caregivers = self.get_available_caregivers(day, shift, "preferred")
                if preferred_caregivers:
                    assigned = preferred_caregivers[0]
                else:
                    # If no 'preferred' caregivers, check for 'available' caregivers
                    available_caregivers = self.get_available_caregivers(day, shift, "available")
                    assigned = available_caregivers[0] if available_caregivers else None
                
                self.schedule[day][shift] = assigned
                if assigned:
                    # Update hours assigned for pay calculation
                    assigned.hours_assigned += 6  # Each shift is 6 hours

    def get_available_caregivers(self, day, shift, status):
        """
        Returns a list of caregivers available for a specific day, shift, and status.
        """
        return [
            caregiver for caregiver in self.caregivers
            if day in caregiver.availability and caregiver.availability[day].get(shift) == status
        ]

    def display_schedule(self):
        """
        Displays the schedule in the terminal.
        """
        for day, shifts in self.schedule.items():
            print(f"Schedule for Day {day}:")
            for shift, caregiver in shifts.items():
                caregiver_name = caregiver.name if caregiver else "No caregiver assigned"
                print(f"  {shift}: {caregiver_name}")

    def generate_html_schedule(self, filename="schedule.html", days_in_month=31, first_day_offset=0):
        """
        Generates an HTML file to display the schedule in a weekly layout.
        """
        html_content = """
        <html>
        <head>
            <title>Work Schedule</title>
            <style>
                table {
                    border-collapse: collapse;
                    width: 100%;
                    margin: 20px 0;
                }
                th, td {
                    border: 1px solid black;
                    padding: 10px;
                    text-align: center;
                }
                th {
                    background-color: #f2f2f2;
                }
                td {
                    height: 100px;
                    vertical-align: top;
                }
            </style>
        </head>
        <body>
            <h1>Work Schedule</h1>
            <table>
                <tr>
                    <th>Mon</th>
                    <th>Tue</th>
                    <th>Wed</th>
                    <th>Thu</th>
                    <th>Fri</th>
                    <th>Sat</th>
                    <th>Sun</th>
                </tr>
        """

        current_day = 1
        for week in range(6):  # up to 6 weeks in a month
            html_content += "<tr>"
            for day in range(7):  # 7 days per week
                if week == 0 and day < first_day_offset or current_day > days_in_month:
                    html_content += "<td></td>"  # empty cell for padding
                else:
                    shifts = self.schedule[current_day]
                    am_shift = shifts["7AM-1PM"].name if shifts["7AM-1PM"] else "No caregiver assigned"
                    pm_shift = shifts["1PM-7PM"].name if shifts["1PM-7PM"] else "No caregiver assigned"
                    html_content += f"<td>{current_day}<br><b>AM:</b> {am_shift}<br><b>PM:</b> {pm_shift}</td>"
                    current_day += 1
            html_content += "</tr>"

        html_content += """
            </table>
        </body>
        </html>
        """

        with open(filename, "w") as html_file:
            html_file.write(html_content)
        print(f"HTML schedule saved as {filename}")

    def generate_pay_report(self, filename="pay_report.txt"):
        """
        Generates a pay report for all caregivers.
        """
        total_weekly = 0
        total_monthly = 0
        with open(filename, "w") as file:
            file.write("Weekly Pay Report\n")
            file.write("=================\n")
            for caregiver in self.caregivers:
                weekly_pay = caregiver.calculate_weekly_pay()
                total_weekly += weekly_pay
                total_monthly += weekly_pay * 4
                file.write(f"{caregiver.name}: ${weekly_pay:.2f}/week\n")
            file.write("\n")
            file.write(f"Total Weekly Pay: ${total_weekly:.2f}\n")
            file.write(f"Total Monthly Pay: ${total_monthly:.2f}\n")
        print(f"Pay report saved as {filename}")


# main
if __name__ == "__main__":
    # create caregiver instances with availability
    caregiver1 = Caregiver("Abby", "family", {1: {"7AM-1PM": "preferred", "1PM-7PM": "available"}})
    caregiver2 = Caregiver("Madison", "paid", {1: {"7AM-1PM": "available", "1PM-7PM": "preferred"}})
    caregiver3 = Caregiver("Luke", "family", {4: {"7AM-1PM": "preferred", "1PM-7PM": "unavailable"}})

    caregivers = [caregiver1, caregiver2, caregiver3]

    # initialize schedule
    schedule = Schedule(caregivers)
    schedule.initialize_schedule()

    # assign caregivers
    schedule.assign_caregivers()

    # display final schedule
    schedule.display_schedule()

    # generate HTML file for schedule
    schedule.generate_html_schedule(first_day_offset=2)  # October 2024 starts on a Tuesday

    # generate pay report
    schedule.generate_pay_report()


Schedule for Day 1:
  7AM-1PM: Abby
  1PM-7PM: Madison
Schedule for Day 2:
  7AM-1PM: No caregiver assigned
  1PM-7PM: Abby
Schedule for Day 3:
  7AM-1PM: Madison
  1PM-7PM: No caregiver assigned
Schedule for Day 4:
  7AM-1PM: Luke
  1PM-7PM: Luke
Schedule for Day 5:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 6:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 7:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 8:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 9:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 10:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 11:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 12:
  7AM-1PM: No caregiver assigned
  1PM-7PM: No caregiver assigned
Schedule for Day 13:
  7AM-1PM: No caregiver assigned
  1PM-7